# Load Dataset and re-process

In [3]:

from datasets import load_dataset


/home/fenghe/anaconda3/envs/RL4LM3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the SQuAD dataset
dataset = load_dataset('squad')


# Different models training

## -t5-small

In [18]:
def preprocess_t5_data(example):
    if example['answers']['text']:
        answer_text = example['answers']['text'][0]
    else:
        answer_text = "No answer found"
    return {
        'input_text': f"answer: {answer_text}",
        'target_text': f"question: {example['question']}"  # T5需要任务前缀
    }

processed_t5_dataset = dataset.map(preprocess_t5_data)

Map: 100%|██████████| 10570/10570 [00:00<00:00, 31216.43 examples/s]


In [19]:
train_test_split_t5 = processed_t5_dataset['train'].train_test_split(test_size=0.1)
train_dataset_t5 = train_test_split_t5['train']
val_dataset_t5 = train_test_split_t5['test']

In [20]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset_t5 = train_dataset_t5.map(tokenize_t5_function, batched=True)
tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)

loading file https://huggingface.co/t5-small/resolve/main/spiece.model from cache at /home/fenghe/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-small/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/t5-small/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/t5-small/resolve/main/tokenizer_config.json from cache at /home/fenghe/.cache/huggingface/transformers/edbdb128668f5837a316e446d9d0dd59018a797db29e5bb3652db0e8fbe9cda5.679fa4e712151a8d260bfc3f42ace42f9309fe985622073b432663164029e77e
Map: 100%|██████████| 8760/8760 [00:01<00:00, 5649.20 examples/s]


In [21]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t5_model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)


loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/fenghe/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.d67b370cd9d75f81ad4eb421ee7b8db09e0b6a6c693b8c2b423af5d7bcac6205
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "

In [22]:
from transformers import Trainer, TrainingArguments

training_args_t5 = TrainingArguments(
    output_dir='./results_t5_small',
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    report_to="none"
)

trainer_t5 = Trainer(
    model=t5_model,
    args=training_args_t5,
    train_dataset=tokenized_train_dataset_t5,
    eval_dataset=tokenized_val_dataset_t5
)


PyTorch: setting up devices


In [23]:

trainer_t5.train()


The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: context, answers, target_text, input_text, id, question, title. If context, answers, target_text, input_text, id, question, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/fenghe/anaconda3/envs/RL4LM3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 78839
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 59130
  1%|          | 500/59130 [00:14<29:23, 33.25it/s

{'loss': 0.5892, 'learning_rate': 0.0002974632166412988, 'epoch': 0.03}


  2%|▏         | 1000/59130 [00:30<29:14, 33.14it/s] Saving model checkpoint to ./results_t5_small/checkpoint-1000
Configuration saved in ./results_t5_small/checkpoint-1000/config.json
Model weights saved in ./results_t5_small/checkpoint-1000/pytorch_model.bin


{'loss': 0.4387, 'learning_rate': 0.0002949264332825976, 'epoch': 0.05}


  3%|▎         | 1500/59130 [00:45<28:28, 33.72it/s]  Saving model checkpoint to ./results_t5_small/checkpoint-1500
Configuration saved in ./results_t5_small/checkpoint-1500/config.json
Model weights saved in ./results_t5_small/checkpoint-1500/pytorch_model.bin


...

Deleting older checkpoint [results_t5_small/checkpoint-57000] due to args.save_total_limit
100%|█████████▉| 59000/59130 [30:50<00:03, 33.59it/s]Saving model checkpoint to ./results_t5_small/checkpoint-59000
Configuration saved in ./results_t5_small/checkpoint-59000/config.json
Model weights saved in ./results_t5_small/checkpoint-59000/pytorch_model.bin


{'loss': 0.3621, 'learning_rate': 6.595636732623033e-07, 'epoch': 2.99}


Deleting older checkpoint [results_t5_small/checkpoint-57500] due to args.save_total_limit
100%|█████████▉| 59127/59130 [30:54<00:00, 33.68it/s]The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: context, answers, target_text, input_text, id, question, title. If context, answers, target_text, input_text, id, question, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8760
  Batch size = 4
                                                     
100%|██████████| 59130/59130 [31:12<00:00, 33.68it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 59130/59130 [31:12<00:00, 31.57it/s]

{'eval_loss': 0.359306275844574, 'eval_runtime': 17.7606, 'eval_samples_per_second': 493.228, 'eval_steps_per_second': 123.307, 'epoch': 3.0}
{'train_runtime': 1872.6941, 'train_samples_per_second': 126.298, 'train_steps_per_second': 31.575, 'train_loss': 0.3820187634307278, 'epoch': 3.0}


TrainOutput(global_step=59130, training_loss=0.3820187634307278, metrics={'train_runtime': 1872.6941, 'train_samples_per_second': 126.298, 'train_steps_per_second': 31.575, 'train_loss': 0.3820187634307278, 'epoch': 3.0})

In [24]:
def generate_question(answer):
    t5_model.eval()  # 将模型设置为评估模式
    input_ids = t5_tokenizer.encode("answer: " + answer, return_tensors="pt").to(device)
    outputs = t5_model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [25]:
# 获取几个样本
samples = val_dataset_t5.shuffle(seed=42).select(range(5))  # 随机选择5个样本

# 生成问题并比较
for example in samples:
    generated_question = generate_question(example['input_text'].replace("answer: ", ""))
    print(f"Answer: {example['input_text'].replace('answer: ', '')}")
    print(f"Generated Question: {generated_question}")
    print(f"Actual Question: {example['target_text']}\n")


Answer: Habitat Conservation Plan
Generated Question: question: What is the name of the plan that aims to protect wildlife from extinction?
Actual Question: question: What program gives incentives to private landowners to protect species on their land?

Answer: Vedantins
Generated Question: question: What is the name of the sulfate that can be used to produce uranium?
Actual Question: question: What school thought that language was supposed to be widened to describe and develop?

Answer: the Bagratid Dynasty
Generated Question: question: What was the name of the dynasty that led to the end of the Ottoman empire?
Actual Question: question: What dynasty was Ashot I part of?

Answer: RES Directive
Generated Question: question: What is the name of the directive that governs the use of uranium uranium?
Actual Question: question: What states that EU Member States must ensure that the origin of electricity produced from renewables can be guaranteed?

Answer: modern-day Eritrea
Generated Quest

## -bart-large

In [ ]:

def preprocess_squad(example):
    # Invert the dataset by treating the answer as input and the question as output
    if example['answers']['text']:
    # SQuAD has answers as a list of possible answer texts; we'll just use the first one for simplicity
        answer_text = example['answers']['text'][0]
    else:
        answer_text = "No answer found"
    return {
        'input_text': f"answer: {answer_text}",
        'target_text': example['question']
    }


# Preprocess the dataset
processed_dataset = dataset.map(preprocess_squad)

Map: 100%|██████████| 10570/10570 [00:00<00:00, 30497.21 examples/s]


In [ ]:
train_test_split = processed_dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [ ]:
print(train_dataset.shape)
print(val_dataset.shape)

(78839, 7)
(8760, 7)


In [7]:
from transformers import BartTokenizer

bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

def tokenize_bart_function(examples):
    model_inputs = bart_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = bart_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset_bart = train_dataset.map(tokenize_bart_function, batched=True)
tokenized_val_dataset_bart = val_dataset.map(tokenize_bart_function, batched=True)

Map: 100%|██████████| 8760/8760 [00:02<00:00, 4341.60 examples/s]


In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import BartForConditionalGeneration

bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to(device)


In [12]:
from transformers import Trainer, TrainingArguments

training_args_bart = TrainingArguments(
    output_dir='./results_bart_2',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    report_to="none"
)

trainer_bart = Trainer(
    model=bart_model,
    args=training_args_bart,
    train_dataset=tokenized_train_dataset_bart,
    eval_dataset=tokenized_val_dataset_bart
)


PyTorch: setting up devices


In [13]:
trainer_bart.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: context, answers, target_text, input_text, id, question, title. If context, answers, target_text, input_text, id, question, title are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/home/fenghe/anaconda3/envs/RL4LM3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 78839
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 59130
  1%|          | 500/59130 [01:07<2:11:36,  7.

{'loss': 1.6307, 'learning_rate': 4.9577202773549806e-05, 'epoch': 0.03}


Model weights saved in ./results_bart_2/checkpoint-500/pytorch_model.bin
  2%|▏         | 1000/59130 [02:30<2:11:24,  7.37it/s]Saving model checkpoint to ./results_bart_2/checkpoint-1000
Configuration saved in ./results_bart_2/checkpoint-1000/config.json


{'loss': 0.4493, 'learning_rate': 4.9154405547099616e-05, 'epoch': 0.05}


Model weights saved in ./results_bart_2/checkpoint-1000/pytorch_model.bin
  3%|▎         | 1500/59130 [04:12<2:09:45,  7.40it/s] Saving model checkpoint to ./results_bart_2/checkpoint-1500
Configuration saved in ./results_bart_2/checkpoint-1500/config.json


...

Model weights saved in ./results_bart_2/checkpoint-58500/pytorch_model.bin
Deleting older checkpoint [results_bart_2/checkpoint-57000] due to args.save_total_limit
100%|█████████▉| 59000/59130 [2:33:17<00:17,  7.40it/s]Saving model checkpoint to ./results_bart_2/checkpoint-59000
Configuration saved in ./results_bart_2/checkpoint-59000/config.json


{'loss': 0.2307, 'learning_rate': 1.0992727887705057e-07, 'epoch': 2.99}


Model weights saved in ./results_bart_2/checkpoint-59000/pytorch_model.bin
Deleting older checkpoint [results_bart_2/checkpoint-57500] due to args.save_total_limit
100%|██████████| 59130/59130 [2:33:42<00:00,  7.70it/s]The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: context, answers, target_text, input_text, id, question, title. If context, answers, target_text, input_text, id, question, title are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8760
  Batch size = 4
                                                       
100%|██████████| 59130/59130 [2:35:03<00:00,  7.70it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 59130/59130 [2:35:03<00:00,  6.36it/s]

{'eval_loss': 0.33391496539115906, 'eval_runtime': 81.0456, 'eval_samples_per_second': 108.087, 'eval_steps_per_second': 27.022, 'epoch': 3.0}
{'train_runtime': 9303.3948, 'train_samples_per_second': 25.423, 'train_steps_per_second': 6.356, 'train_loss': 0.3114789662779678, 'epoch': 3.0}


TrainOutput(global_step=59130, training_loss=0.3114789662779678, metrics={'train_runtime': 9303.3948, 'train_samples_per_second': 25.423, 'train_steps_per_second': 6.356, 'train_loss': 0.3114789662779678, 'epoch': 3.0})

In [16]:
def generate_question(answer):
    bart_model.eval()  # 将模型设置为评估模式
    input_ids = bart_tokenizer.encode("answer: " + answer, return_tensors="pt").to(device)
    outputs = bart_model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    question = bart_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [17]:
# 获取几个样本
samples = val_dataset.shuffle(seed=42).select(range(5))  # 随机选择5个样本

# 生成问题并比较
for example in samples:
    generated_question = generate_question(example['input_text'].replace("answer: ", ""))
    print(f"Answer: {example['input_text'].replace('answer: ', '')}")
    print(f"Generated Question: {generated_question}")
    print(f"Actual Question: {example['target_text']}\n")


Answer: power was transferred to the eldest member
Generated Question: What happened to the power of the members of the upper house after the election?
Actual Question: What did the rota system do?

Answer: the Bipartisan Campaign Reform Act of 2002
Generated Question: What act was passed in 2002 to reform campaign spending?
Actual Question: What finance act affected the 2004 election?

Answer: directly attached to the end of the motor
Generated Question: Where is the rotor located in a synchronous motor?
Actual Question: In a gearless traction engine, what is the drive sheave attached to?

Answer: Neolithic cave-burial
Generated Question: What is the earliest evidence of human activity in Myanmar?
Actual Question: What kind of bural was at Adaoutse, Bouches-du-Rhône?

Answer: 960–1279
Generated Question: When did the Tang dynasty rule?
Actual Question: When did the Song dynasty take place?



## -t5-base

In [26]:
def preprocess_t5_data(example):
    if example['answers']['text']:
        answer_text = example['answers']['text'][0]
    else:
        answer_text = "No answer found"
    return {
        'input_text': f"answer: {answer_text}",
        'target_text': f"question: {example['question']}"
    }

processed_t5_dataset = dataset.map(preprocess_t5_data)

In [27]:
train_test_split_t5 = processed_t5_dataset['train'].train_test_split(test_size=0.1)
train_dataset_t5 = train_test_split_t5['train']
val_dataset_t5 = train_test_split_t5['test']

In [28]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset_t5 = train_dataset_t5.map(tokenize_t5_function, batched=True)
tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)


loading file https://huggingface.co/t5-base/resolve/main/spiece.model from cache at /home/fenghe/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/fenghe/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start

In [29]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t5_base_model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)


loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/fenghe/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
     

In [30]:
from transformers import Trainer, TrainingArguments

training_args_t5 = TrainingArguments(
    output_dir='./results_t5_base',
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    report_to="none"
)

trainer_t5 = Trainer(
    model=t5_base_model,
    args=training_args_t5,
    train_dataset=tokenized_train_dataset_t5,
    eval_dataset=tokenized_val_dataset_t5
)


PyTorch: setting up devices


In [31]:

trainer_t5.train()


The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: context, answers, target_text, input_text, id, question, title. If context, answers, target_text, input_text, id, question, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/fenghe/anaconda3/envs/RL4LM3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 78839
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 59130
  1%|          | 500/59130 [00:46<1:30:36, 10.79it

{'loss': 0.5303, 'learning_rate': 0.0002974632166412988, 'epoch': 0.03}


Model weights saved in ./results_t5_base/checkpoint-500/pytorch_model.bin
  2%|▏         | 1000/59130 [01:33<1:29:28, 10.83it/s]Saving model checkpoint to ./results_t5_base/checkpoint-1000
Configuration saved in ./results_t5_base/checkpoint-1000/config.json


{'loss': 0.389, 'learning_rate': 0.0002949264332825976, 'epoch': 0.05}


Model weights saved in ./results_t5_base/checkpoint-1000/pytorch_model.bin
  3%|▎         | 1500/59130 [02:21<1:28:48, 10.82it/s]Saving model checkpoint to ./results_t5_base/checkpoint-1500
Configuration saved in ./results_t5_base/checkpoint-1500/config.json


...

Model weights saved in ./results_t5_base/checkpoint-58500/pytorch_model.bin
Deleting older checkpoint [results_t5_base/checkpoint-57000] due to args.save_total_limit
100%|█████████▉| 59000/59130 [1:36:17<00:12, 10.82it/s]Saving model checkpoint to ./results_t5_base/checkpoint-59000
Configuration saved in ./results_t5_base/checkpoint-59000/config.json


{'loss': 0.2813, 'learning_rate': 6.595636732623033e-07, 'epoch': 2.99}


Model weights saved in ./results_t5_base/checkpoint-59000/pytorch_model.bin
Deleting older checkpoint [results_t5_base/checkpoint-57500] due to args.save_total_limit
100%|█████████▉| 59129/59130 [1:36:30<00:00, 10.84it/s]The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: context, answers, target_text, input_text, id, question, title. If context, answers, target_text, input_text, id, question, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8760
  Batch size = 4
                                                       
100%|██████████| 59130/59130 [1:37:25<00:00, 10.84it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 59130/59130 [1:37:25<00:00, 10.11it/s]

{'eval_loss': 0.3221367597579956, 'eval_runtime': 54.8193, 'eval_samples_per_second': 159.798, 'eval_steps_per_second': 39.949, 'epoch': 3.0}
{'train_runtime': 5845.8802, 'train_samples_per_second': 40.459, 'train_steps_per_second': 10.115, 'train_loss': 0.32119348114080765, 'epoch': 3.0}


TrainOutput(global_step=59130, training_loss=0.32119348114080765, metrics={'train_runtime': 5845.8802, 'train_samples_per_second': 40.459, 'train_steps_per_second': 10.115, 'train_loss': 0.32119348114080765, 'epoch': 3.0})

In [32]:
def generate_question(answer):
    t5_base_model.eval()  
    input_ids = t5_tokenizer.encode("answer: " + answer, return_tensors="pt").to(device)
    outputs = t5_base_model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [33]:
# 获取几个样本
samples = val_dataset.shuffle(seed=42).select(range(5))  # 随机选择5个样本

# 生成问题并比较
for example in samples:
    generated_question = generate_question(example['input_text'].replace("answer: ", ""))
    print(f"Answer: {example['input_text'].replace('answer: ', '')}")
    print(f"Generated Question: {generated_question}")
    print(f"Actual Question: {example['target_text']}\n")


Answer: power was transferred to the eldest member
Generated Question: question: What happens when a member of a federation loses power?
Actual Question: What did the rota system do?

Answer: the Bipartisan Campaign Reform Act of 2002
Generated Question: question: What was the name of the bill that was passed by Congress in 2002?
Actual Question: What finance act affected the 2004 election?

Answer: directly attached to the end of the motor
Generated Question: question: How is the servo motor mounted?
Actual Question: In a gearless traction engine, what is the drive sheave attached to?

Answer: Neolithic cave-burial
Generated Question: question: What type of burial was held at the site?
Actual Question: What kind of bural was at Adaoutse, Bouches-du-Rhône?

Answer: 960–1279
Generated Question: question: What was the span of the Han dynasty?
Actual Question: When did the Song dynasty take place?

